In [ ]:
# importing necessary libraries 

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision 
from torchvision import transforms, datasets, models 
from torch.nn import functional as F 
from PIL import Image
import pandas as pd 
import numpy as np
import os
import sys
import glob
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt 
import cv2
from sklearn.preprocessing import StandardScaler
from skimage.transform import rotate, AffineTransform
import random
from scipy import ndimage
import openslide
import matplotlib.patches as patches
from matplotlib.patches import Polygon
from collections import Counter
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc, classification_report
import seaborn as sns 
import gc 
import math 
import collections 
from collections import defaultdict 

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') 

In [ ]:
# dictionary for width and height 
dimensions_dict = {'A01': (62625, 42113), 'A02': (54721, 46305), 'A03': (57345, 38945), 'A04': (64703, 45808), 
                   'A05': (51553, 42625), 'A06': (62625, 36833), 'A07': (50529, 36833), 'A08': (58401, 41057), 
                   'A09': (64191, 34700), 'A10': (58401, 39959)} 

**OBTAINING PATCHES AND PATCH LEVEL LABELS** 

In [ ]:
# Function to obtain patches and their labels against annotation matrices 

def get_all_patches_info(df): 
    
    wsi_nos = df['WSI No.'].to_list() 
    x_pixels = df['X Pixel'].to_list() 
    y_pixels = df['Y Pixel'].to_list() 
    updated_labels = df['Exact Label'].to_list() 
    patch_paths = ['/kaggle/input/bach-complete-patches/all_wsi_patches_224/' + str(wsi_nos[i]) + '_' + str(x_pixels[i]) + '_' + str(y_pixels[i]) + '.jpg' for i in range(len(wsi_nos))] 

    return wsi_nos, patch_paths, updated_labels, x_pixels, y_pixels 

In [ ]:
# Fold 1:
#   Train: ['A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10']
#   Test : ['A01', 'A02']
# Fold 2:
#   Train: ['A01', 'A02', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10']
#   Test : ['A03', 'A04']
# Fold 3:
#   Train: ['A01', 'A02', 'A03', 'A04', 'A07', 'A08', 'A09', 'A10']
#   Test : ['A05', 'A06']
# Fold 4:
#   Train: ['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A09', 'A10']
#   Test : ['A07', 'A08']
# Fold 5:
#   Train: ['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08']
#   Test : ['A09', 'A10']

In [ ]:
# this example is for the fifth fold 

df = pd.read_csv('patch_info.csv') 

train_df1 = df[df['WSI No.'] == 'A01'] 
train_df2 = df[df['WSI No.'].isin(['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08'])] 
val_df = df[df['WSI No.'].isin(['A09', 'A10'])] 
test_df =  df[df['WSI No.'].isin(['A09', 'A10'])] 

In [ ]:
# obtaining patch infos for all wsis (train) 

train_wsi_nos_1, train_patch_paths_1, train_updated_labels_1, train_x_pixels_1, train_y_pixels_1 = get_all_patches_info(train_df1) 
train_wsi_nos_2, train_patch_paths_2, train_updated_labels_2, train_x_pixels_2, train_y_pixels_2 = get_all_patches_info(train_df2) 

In [ ]:
# obtaining patch infos for all wsis (val) 

val_wsi_nos, val_patch_paths, val_updated_labels_temp, val_x_pixels, val_y_pixels = get_all_patches_info(val_df) 

In [ ]:
# obtaining patch infos for all wsis (test) 

test_wsi_nos, test_patch_paths, test_updated_labels_temp, test_x_pixels, test_y_pixels = get_all_patches_info(test_df) 

In [ ]:
# filtering only normal / background patches (train) 

train_wsi_nos = [] 
train_patch_paths = [] 
train_updated_labels_temp = [] 
train_x_pixels = [] 
train_y_pixels = [] 

for i in range(len(train_wsi_nos_1)): 
    train_wsi_nos.append(train_wsi_nos_1[i]) 
    train_patch_paths.append(train_patch_paths_1[i]) 
    train_updated_labels_temp.append(train_updated_labels_1[i]) 
    train_x_pixels.append(train_x_pixels_1[i]) 
    train_y_pixels.append(train_y_pixels_1[i]) 

for i in range(len(train_wsi_nos_2)): 
    if train_updated_labels_2[i] != 'White': 
        train_wsi_nos.append(train_wsi_nos_2[i]) 
        train_patch_paths.append(train_patch_paths_2[i]) 
        train_updated_labels_temp.append(train_updated_labels_2[i]) 
        train_x_pixels.append(train_x_pixels_2[i]) 
        train_y_pixels.append(train_y_pixels_2[i]) 

In [ ]:
# Encode with binary class labels

label_to_onehot = {
    'Benign': 0,
    'White': 0
}

# Mark 'Benign' and 'White' as 0, any other label as 1
train_updated_labels = [label_to_onehot.get(label, 1) for label in train_updated_labels_temp] 
val_updated_labels = [label_to_onehot.get(label, 1) for label in val_updated_labels_temp] 
test_updated_labels = [label_to_onehot.get(label, 1) for label in test_updated_labels_temp] 

**CLASS DISTRIBUTION**

In [ ]:
# Count label occurrences
train_counts = collections.Counter(train_updated_labels)
val_counts = collections.Counter(val_updated_labels)
test_counts = collections.Counter(test_updated_labels)

# Create a 1x3 subplot figure
fig, axes = plt.subplots(1, 3, figsize=(18, 5), sharey=True)

# Common settings
datasets = ['Train', 'Validation', 'Test']
counts_list = [train_counts, val_counts, test_counts]
colors = ['skyblue', 'salmon']
labels = ['Benign / White (0)', 'Carcinoma (1)']

for ax, counts, title in zip(axes, counts_list, datasets):
    ax.bar(counts.keys(), counts.values(), color=colors)
    ax.set_xticks([0, 1])
    ax.set_xticklabels(labels, rotation=15)
    ax.set_ylabel('Count')
    ax.set_title(f'Distribution of Updated Labels - {title}')
    ax.grid(axis='y', linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

**CUSTOM DATASET & DATA LOADER FOR TESTING** 

In [ ]:
from torchvision.transforms import ToTensor

class CustomDataset(Dataset):
    def __init__(self, patches, labels):
        assert len(patches) == len(labels), "Mismatch in number of patches and labels"
        self.patches = patches
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Check for valid index range
        if idx >= len(self.labels) or idx >= len(self.patches):
            raise IndexError(f"Index {idx} out of range for dataset with length {len(self)}")

        patch = Image.open(self.patches[idx]).convert("RGB")  # Adjust this if `patches` are file paths
        patch = patch.resize((224, 224))  
        patch = ToTensor()(patch)
        patch = nn.functional.normalize(patch, dim=0, p=2)

        label = torch.tensor(self.labels[idx], dtype=torch.long)
        
        return patch, label

In [ ]:
# Create instances of custom datasets

train_dataset = CustomDataset(train_patch_paths, train_updated_labels) 
val_dataset = CustomDataset(val_patch_paths, val_updated_labels) 
test_dataset = CustomDataset(test_patch_paths, test_updated_labels) 

In [ ]:
batch_size = 128 

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers = 8) 
valid_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers = 8) 
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers = 8) 

**MODEL (USING IMPORTED WEIGHTS FROM LAN** 

In [ ]:
from torch.nn import functional as F
import torchvision.models as models

class Pipeline(nn.Module):
    def __init__(self,num_classes):
        #define necessary layers
        super().__init__()
        self.num_classes = num_classes
        self.base = models.vit_b_32(weights='DEFAULT') 

        # unfreeze model weights
        for param in self.base.parameters():
            param.requires_grad = True
        
        self.flatten = nn.Flatten()
        
        self.head = nn.Sequential(
            nn.Linear(1000, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        ) 
        
        # freeze head weights
        '''
        for param in self.head.parameters():
            param.requires_grad = False
        '''
        
    def forward(self,X):
            X = self.base(X)
            X = self.flatten(X)
            X = self.head(X)
            return X, F.sigmoid(X)
    
pos_weight = torch.tensor([0.0]) 
pos_weight = pos_weight.to(device)
loss_fn = nn.BCEWithLogitsLoss(pos_weight = pos_weight)
model = Pipeline(1)

In [ ]:
# Check if GPU is available
if torch.cuda.is_available():
    model = model.to('cuda')
    print('available')

#criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(), lr=0.0)

In [ ]:
!export CUDA_LAUNCH_BLOCKING=1

**FINE-TUNING ON WHITE AREA / NORMAL REGION FOR TWO WSIS**

In [ ]:
# Initialize empty lists to store loss and accuracy for training and validation
train_losses = []
valid_losses = []
train_accuracies = []
valid_accuracies = []
roc_values_train = []
roc_values_val = []

In [ ]:
from sklearn.metrics import roc_curve, auc, classification_report
import math  

# Training loop
num_epochs = 0  # Adjust as needed 

for epoch in range(num_epochs):
    model.train()
    y_true_train = []
    y_scores_train = []
    train_loss = 0
    correct = 0
    total = 0
    for images, labels in tqdm(train_loader):
        # Move data to GPU if available
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs_without_sigmoid, outputs_with_sigmoid = model(images)
        loss = loss_fn(outputs_without_sigmoid.squeeze(-1), labels.float())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()  # L+=l.item()
        predicted = (outputs_with_sigmoid >= 0.5).long().squeeze(-1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        y_true_train.extend(labels.cpu().detach().numpy())
        y_scores_train.extend(outputs_with_sigmoid.cpu().detach().numpy())

    train_losses.append(train_loss / len(train_loader))
    train_accuracies.append(100 * correct / total)

    # roc auc logic
    fpr_train, tpr_train, _ = roc_curve(y_true_train, y_scores_train)
    roc_auc_train = auc(fpr_train, tpr_train)
    roc_values_train.append(roc_auc_train)

    # Plot ROC curve for the training set

    # Validate your model after each epoch if needed
    model.eval()
    valid_loss = 0
    correct = 0
    total = 0
    y_true_val = []
    y_scores_val = []
    with torch.no_grad():
        for images, labels in tqdm(valid_loader):
            # Move data to GPU if available
            images = images.to(device)
            labels = labels.to(device)
            outputs_without_sigmoid, outputs_with_sigmoid = model(images)
            loss = loss_fn(outputs_without_sigmoid.squeeze(-1), labels.float())
            valid_loss += loss.item()  # L+=l.item()
            predicted = (outputs_with_sigmoid >= 0.5).long().squeeze(-1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            y_true_val.extend(labels.cpu().detach().numpy())
            y_scores_val.extend(outputs_with_sigmoid.cpu().detach().numpy())

    valid_losses.append(valid_loss / len(valid_loader))
    valid_accuracies.append(100 * correct / total)
    # roc auc logic
    fpr_val, tpr_val, _ = roc_curve(y_true_val, y_scores_val)
    roc_auc_val = auc(fpr_val, tpr_val)
    roc_values_val.append(roc_auc_val)

    # Classification report
    print(f'Epoch {epoch + 1}, Train Accuracy: {train_accuracies[-1]:.2f}%, Train Loss: {train_losses[-1]:.2f}%, Train ROC-AUC: {roc_auc_train:.2f}%, Val Accuracy: {valid_accuracies[-1]:.2f}%, Val Loss: {valid_losses[-1]:.2f}%, Val ROC-AUC: {roc_auc_val:.2f}%')

    y_true_val = np.array(y_true_val)
    y_pred_val = (np.array(y_scores_val) >= 0.5).astype(int)
    print("Validation Classification Report:")
    print(classification_report(y_true_val, y_pred_val))

In [ ]:
def display_training_curves(train_losses, valid_losses, train_accuracies, valid_accuracies, subplot):
    if subplot % 10 == 1:  # set up the subplots on the first call
        plt.subplots(figsize=(12, 5), facecolor='#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(train_losses)
    ax.plot(valid_losses)
    ax.set_title('Model Loss')
    ax.set_ylabel('Loss')
    ax.set_xlabel('Epoch')
    ax.legend(['Train', 'Valid'])

    ax = plt.subplot(subplot + 1)
    ax.set_facecolor('#F8F8F8')
    ax.plot(train_accuracies)
    ax.plot(valid_accuracies)
    ax.set_title('Model Accuracy')
    ax.set_ylabel('Accuracy')
    ax.set_xlabel('Epoch')
    ax.legend(['Train', 'Valid']) 

In [ ]:
# Plot the training curves
display_training_curves(train_losses, valid_losses, train_accuracies, valid_accuracies, 121)

**EVALUATION**

In [ ]:
model.eval()

test_loss = 0
correct = 0
total = 0
y_true_test = []
y_pred_test = []
y_probs = []

with torch.no_grad():
    for images, labels in tqdm(test_loader):
        # Move data to GPU if available
        images = images.to(device)
        labels = labels.to(device)
        outputs_without_sigmoid, outputs_with_sigmoid = model(images)
        loss = loss_fn(outputs_without_sigmoid.squeeze(-1), labels.float())
        test_loss += loss.item()  # L+=l.item()
        predicted = (outputs_with_sigmoid >= 0.5).long().squeeze(-1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        y_true_test.extend(labels.cpu().detach().numpy())
        y_pred_test.extend(predicted.cpu().detach().numpy())
        y_probs.extend(outputs_with_sigmoid.cpu().detach().numpy())

test_loss /= len(test_loader)
test_accuracy = 100 * correct / total

# Calculate ROC-AUC for test set
fpr_test, tpr_test, _ = roc_curve(y_true_test, y_probs)
roc_auc_test = auc(fpr_test, tpr_test)

print(f'Test Accuracy: {test_accuracy:.2f}%, Test Loss: {test_loss:.2f}%')
print(f'Test ROC-AUC: {roc_auc_test:.4f}')

# Classification report for test set
print("Test Classification Report:")
print(classification_report(y_true_test, y_pred_test)) 

In [ ]:
# ROC curve for the test set
plt.figure(figsize=(4, 4))
plt.plot(fpr_test, tpr_test, label=f'Test ROC Curve (AUC = {roc_auc_test:.2f})')
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve - Test Set')
plt.legend()
plt.show()

In [ ]:
# Generate confusion matrix
conf_matrix = confusion_matrix(y_true_test, y_pred_test)

# Plot confusion matrix
plt.figure(figsize=(5, 5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap="Blues", cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Patch Level Confusion Matrix')
plt.show()

**SAVING UPDATED MODEL WEIGHTS** 

In [ ]:
# Saving updated model weights 

model.cpu() 
torch.save(model.state_dict(), 'model_weights.pth') 